In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
from nltk.stem.porter import *
# For Building the model
from sklearn.model_selection import train_test_split
import tensorflow as tf
import seaborn as sns

#For data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

pd.options.plotting.backend = "plotly"

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv
/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv
/kaggle/input/tweeter/tweet test.csv
/kaggle/input/tweeter/tweet train.csv


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
tweet_df = pd.read_csv('/kaggle/input/tweeter/tweet train.csv')
tweet_df.head

<bound method NDFrame.head of           id  label                                              tweet
0          1      0   @user when a father is dysfunctional and is s...
1          2      0  @user @user thanks for #lyft credit i can't us...
2          3      0                                bihday your majesty
3          4      0  #model   i love u take with u all the time in ...
4          5      0             factsguide: society now    #motivation
...      ...    ...                                                ...
31957  31958      0  ate @user isz that youuu?ðððððð...
31958  31959      0    to see nina turner on the airwaves trying to...
31959  31960      0  listening to sad songs on a monday morning otw...
31960  31961      1  @user #sikh #temple vandalised in in #calgary,...
31961  31962      0                   thank you @user for you follow  

[31962 rows x 3 columns]>

In [4]:
tweet_df.shape

(31962, 3)

In [5]:
tweet_df.columns

Index(['id', 'label', 'tweet'], dtype='object')

In [6]:
tweet_df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [7]:
tweet_df.dropna(subset=['tweet','label'],axis=0,inplace=True)

In [8]:
tweet_df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [9]:
X = tweet_df.drop('label',axis=1)
Y = tweet_df['label']

In [10]:
port_stem = PorterStemmer()

In [11]:
def stemming(clean_text):
        stemm_cat = re.sub('[^A-Za-z]',' ',clean_text)
        stemm_cat = stemm_cat.lower()
        stemm_cat = stemm_cat.split()
        stemm_cat = [port_stem.stem(word) for word in stemm_cat if not word in stopwords.words('english')]
        stemm_cat = ' '.join(stemm_cat)
        return stemm_cat

In [12]:
tweet_df['tweet'] = tweet_df['tweet'].apply(stemming)

In [13]:
tweet_df['tweet']

0        user father dysfunct selfish drag kid dysfunct...
1        user user thank lyft credit use caus offer whe...
2                                           bihday majesti
3                              model love u take u time ur
4                                  factsguid societi motiv
                               ...                        
31957                                   ate user isz youuu
31958    see nina turner airwav tri wrap mantl genuin h...
31959             listen sad song monday morn otw work sad
31960     user sikh templ vandalis calgari wso condemn act
31961                                    thank user follow
Name: tweet, Length: 31962, dtype: object

In [14]:
X = tweet_df['tweet'].values
Y = tweet_df['label'].values

In [15]:
X

array(['user father dysfunct selfish drag kid dysfunct run',
       'user user thank lyft credit use caus offer wheelchair van pdx disapoint getthank',
       'bihday majesti', ..., 'listen sad song monday morn otw work sad',
       'user sikh templ vandalis calgari wso condemn act',
       'thank user follow'], dtype=object)

In [16]:
# Converting textual data to numerical data
from sklearn.feature_extraction.text import TfidfVectorizer 

vector = TfidfVectorizer()
vector.fit(X)

TfidfVectorizer()

In [17]:
transform_X = vector.transform(X)

In [18]:
tfidf_array = transform_X.toarray()

print(tfidf_array)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
# Splitting data into train and test

X_train,X_test,Y_train,Y_test = train_test_split(tfidf_array, Y, test_size=0.2, stratify=Y, random_state=42)

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [21]:
model.fit(X_train,Y_train)

LogisticRegression()

In [22]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
model.score(X_test,Y_test)

0.9486938839355545